In [10]:
import zarr
import os
import shutil
import numpy as np
import pandas as pd
import skimage.io
from PIL import Image
from skimage.transform import rescale
from ome_zarr.io import parse_url
from ome_zarr.writer import write_image
from skimage import util


In [ ]:
def import_layer(layer, scale=10, verbose=False):
    """ Import images
    
    Parameters
    --------------
    layer: name of the layer, e.g. pyrmosaic_PolyT
    scale: factor by which the image should be resized
    
    Returns
    --------------
    A np array of shape (Z x Y x X)
    
    """
    
    if verbose: print(f"Import layers for \"{layer}\"")
    # get all files that match the pattern
    ########################################################################
    # IMPORTANT: For the Merscope browser I only imported the 3rd z-stack
    ########################################################################
    files = glob.glob("../MERLIN_output/pictures/" + layer + "_z[3].tif")
    files.sort()
    # import the files and subset, scale=10 -> at 10th of the initial size
    imgs = []
    for f in files:
        if verbose: print(f"... Reading \"{f}\"")
        img = tifffile.imread(f)
        if scale > 1:
            img = img[::scale, ::scale].copy()
        imgs.append(img)
    if verbose: print(f"... stacking")
    imgs = np.stack(imgs, axis=0)
    return imgs[0]


polyT = import_layer("pyrmosaic_PolyT", scale=3, verbose=True)
cell1 = import_layer("pyrmosaic_Cellbound1", scale=3, verbose=True)
cell2 = import_layer("pyrmosaic_Cellbound2", scale=3, verbose=True)
cell3 = import_layer("pyrmosaic_Cellbound3", scale=3, verbose=True)
dapi = import_layer("pyrmosaic_DAPI", scale=3, verbose=True)


stack = np.stack([cell1, cell2, cell3, polyT, dapi], 0)

with tifffile.TiffWriter('full_stack_vitessce.tif',  bigtiff=True) as tif:
    tif.save(stack)

In [11]:
# This is a stack of all th TIFF files from the MERSCOPE output
img = skimage.io.imread(f"full_stack_vitessce.tif")

In [12]:
img.shape

(5, 21673, 19821)

In [13]:
# this is the scale to transport the microns to pixels.
# the value is a result of 
# - the micron_to_mosaic_pixel_transform.csv
# - the factor of how I downsamples the images to save some storage
#   a factor of 3 still gave us pretty good resolution of the images
scale = 3.0865


In [14]:
img

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
   

In [15]:
img.shape

(5, 21673, 19821)

In [16]:
default_window = {
      "start": 0,
      "min": 0,
      "max": 256,
      "end": 100
}


In [19]:
path = "/home/max/Desktop/vitesse/data/image.ome.zarr"
if os.path.exists(path) and os.path.isdir(path):
    shutil.rmtree(path)
os.mkdir(path)
store = parse_url(path, mode="w").store
root = zarr.group(store=store)
write_image(image=img, group=root, axes="cyx", storage_options=dict(chunks=(1, 1024, 1024)))
# optional rendering settings
root.attrs["omero"] = {
    "id": 1,
    "name": 'Images',
    "version": 0.01,
    "rdefs": {
    },      
    "channels": [
        {
            "active": False, # This did not seem to have an effect. Never solved this issues :/
            "coefficient": 1.0,
            "color": "88FF00",
            "family": "linear",
            "inverted": False,
            "label": "Cellboundary 1",
            "window": default_window
        },
        {
            "active": True,
            "coefficient": 1.0,
            "color": "FFFF00",
            "family": "linear",
            "inverted": False,
            "label": "Cellboundary 2",
            "window": default_window
        },
        {
            "active": False,
            "coefficient": 1.0,
            "color": "AE21FF",
            "family": "linear",
            "inverted": False,
            "label": "Cellboundary 3",
            "window": default_window
        },
        {
            "active": True,
            "coefficient": 1.0,
            "color": "0000FF",
            "family": "linear",
            "inverted": False,
            "label": "poly T",
            "window": default_window
        },
        {
            "active": False,
            "coefficient": 1.0,
            "color": "0000FF",
            "family": "linear",
            "inverted": False,
            "label": "DAPI",
            "window": default_window
        }
    ],
}

For some reason, the .zattrs files needs to be adapted. Otherwise Vitessce would not read the files

```
"axes": [
         "c",
         "y",
         "x"
        ],
```